<a href="https://colab.research.google.com/github/KartohaWhy/my_colab/blob/main/Copy_Uplift_Tree_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задача uplift-моделирования

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Olymp_5_2/train.csv')

In [ ]:
df.shape

(297250, 152)

In [ ]:
new_df = df.copy()

In [ ]:
df_without_nan = new_df.copy()

In [ ]:
columns_to_fill_binary = ['feature_1','feature_2','cc_11','cc_12','feature_9','feature_13','bki_19','cc_15','feature_17','feature_21','feature_22','feature_24','feature_25','partner_24',
                          'feature_26']

In [ ]:
for col in columns_to_fill_binary:
    df_without_nan[col] = (new_df[col] >= 0.5).astype(int)
    # df_without_nan[col] = new_df[col].fillna(new_df['successful_utilization'])
    mode_value = df_without_nan[col].mode()[0]
    df_without_nan[col] = new_df[col].fillna(mode_value)

In [ ]:
columns_to_fill_mode = ['mb_1','mb_2','application_2','mb_3','bki_6','application_5','partner_11','bki_12','feature_10','bki_14','partner_14','partner_17',
                        'feature_20','bki_27','application_7','bki_39','application_13','cc_22','application_16','application_15','cc_21','cc_1','feature_6','cc_2','cc_4','cc_6']

In [ ]:
for col in columns_to_fill_mode:
    mode_value = df_without_nan[col].mode()[0]  # Получаем моду (первое значение, если несколько)
    df_without_nan[col] = df_without_nan[col].fillna(mode_value)

In [ ]:
columns_to_fill_median = ['application_1','cc_3','cc_5','cc_7','cc_8','application_2','partner_1','cc_8','mb_4','bki_1','graph_1',
                          'partner_2','bki_2','partner_3','application_3','application_4','feature_3','feature_4','bki_3','bki_4',
                          'bki_5','feature_5','mb_5','cc_9','partner_4','partner_5','cc_10','bki_7','partner_6','partner_7',
                          'bki_8','feature_7','partner_8','bki_9','bki_10','partner_9','partner_10','graph_2','feature_8',
                          'bki_11','mb_6','bki_13','feature_11','partner_12','partner_13']

In [ ]:
excluded_columns = ['successful_utilization', 'treatment'] + columns_to_fill_binary + columns_to_fill_mode

In [ ]:
for col in df_without_nan.columns:
    if col not in excluded_columns and df_without_nan[col].dtype in ['int64', 'float64']:
        median_value = df_without_nan[col].median()
        df_without_nan[col] = df_without_nan[col].fillna(median_value)

In [ ]:
df_without_nan.isna().sum().sort_values(ascending=False).head()

,0
application_1,0
feature_21,0
bki_26,0
bki_27,0
cc_17,0


In [ ]:
RANDOM_STATE=42

In [ ]:
df_ut = df_without_nan.copy()

In [ ]:
df_ut['retro_date'] = pd.to_datetime(df_ut['retro_date'])
df_ut['year'] = df_ut['retro_date'].dt.year
df_ut['month'] = df_ut['retro_date'].dt.month
df_ut['day'] = df_ut['retro_date'].dt.day
df_ut = df_ut.drop('retro_date',axis=1)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import (train_test_split,
                                     cross_validate,
                                     StratifiedKFold)
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
!pip install scikit-uplift

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
from sklift.metrics import uplift_auc_score

from sklift.models import (SoloModel,
                           TwoModels,
                           ClassTransformation,
                           ClassTransformationReg)

from sklearn.impute import SimpleImputer
import lightgbm as lgb
import xgboost as xgb

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.7 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier, Pool

In [ ]:
columns_fi = ['cc_3', 'application_1', 'cc_15', 'application_2', 'feature_1', 'cc_18', 'feature_2', 'mb_1', 'cc_8', 'bki_14', 'cc_2', 'cc_4', 'bki_35', 'bki_3', 'feature_7', 'bki_1', 'month', 'mb_3', 'partner_6', 'feature_20', 'partner_5', 'mb_2', 'application_5', 'bki_7', 'feature_4', 'bki_2', 'partner_9', 'mb_4', 'feature_24', 'bki_27', 'bki_44', 'application_9', 'partner_4', 'partner_10', 'partner_21', 'bki_10', 'bki_26', 'feature_16', 'cc_7', 'cc_13', 'bki_12', 'feature_3', 'mb_5', 'application_4', 'feature_21', 'cc_17', 'partner_1', 'cc_9', 'partner_20', 'application_14', 'application_13', 'partner_15', 'bki_38', 'graph_2', 'bki_18', 'mb_7', 'graph_6', 'bki_25', 'bki_24', 'bki_45', 'bki_21', 'feature_18', 'feature_19', 'cc_5', 'bki_32', 'cc_21', 'application_3', 'graph_5', 'partner_2', 'partner_3', 'partner_23', 'bki_41', 'cc_19', 'mb_8', 'application_12', 'bki_22', 'application_7']

In [ ]:
important_columns = columns_fi + ['treatment', 'successful_utilization']

In [ ]:
df_cb_new = df_ut.copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_ut.drop(columns = 'successful_utilization'),
                                                    df_ut['successful_utilization'],
                                                    test_size=0.25,
                                                    random_state=RANDOM_STATE,
                                                    stratify=df_ut['successful_utilization'])

In [ ]:
categorical_features = ['cc_2', 'cc_4','cc_6']

In [ ]:
X_test.shape

(74313, 78)

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.8 MB/s eta 0:00:00


In [ ]:
import optuna

In [ ]:
def objective(trial):

    param = {
        # 'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),  # максимальная глубина деревьев
        # 'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-5, 100),
        # 'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1),
        # 'random_strength': trial.suggest_loguniform('random_strength', 1e-5, 10),
        # 'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 10, 50),
        'cat_features': categorical_features,  # Укажите свои категориальные функции
        'random_state': trial.suggest_int('random_state', 1, 100),
        'silent': True,
        'iterations': 20,
        # "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        # "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        # "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        # "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
    }

    model = CatBoostClassifier(**param)

    sm = SoloModel(estimator=model)

    sm = sm.fit(X_train.drop(columns='treatment'),
                 y_train,
                 treatment=X_train['treatment'],
                 estimator_fit_params={'cat_features': categorical_features})

    uplift_1 = sm.predict(X_test.drop(columns='treatment'))

    return uplift_auc_score(y_true=y_test,
                             uplift=uplift_1,
                             treatment=X_test['treatment'])

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)
print(study.best_trial)

[I 2024-10-19 19:14:06,815] A new study created in memory with name: no-name-c64ce89c-9deb-4554-8d0e-762b335f4c69
[I 2024-10-19 19:14:21,550] Trial 0 finished with value: 0.022488016206605132 and parameters: {'depth': 9, 'random_state': 40, 'subsample': 0.6325561072042316}. Best is trial 0 with value: 0.022488016206605132.
[I 2024-10-19 19:14:28,798] Trial 1 finished with value: 0.033164528569865154 and parameters: {'depth': 4, 'random_state': 29, 'subsample': 0.9392478243075053}. Best is trial 1 with value: 0.033164528569865154.
[I 2024-10-19 19:14:46,862] Trial 2 finished with value: 0.04466452666314754 and parameters: {'depth': 10, 'random_state': 92, 'subsample': 0.2918700958066203}. Best is trial 2 with value: 0.04466452666314754.
[I 2024-10-19 19:14:56,680] Trial 3 finished with value: 0.026184332477870987 and parameters: {'depth': 6, 'random_state': 86, 'subsample': 0.9336563626154808}. Best is trial 2 with value: 0.04466452666314754.
[I 2024-10-19 19:15:07,931] Trial 4 finished

KeyboardInterrupt: 

In [ ]:
best_model = SoloModel(CatBoostClassifier(random_state=100, depth = 7, subsample = 0.3676697654236052, iterations = 20, cat_features=categorical_features))

In [ ]:
best_model.fit(X_train.drop(columns='treatment'),
                 y_train,
               treatment=X_train['treatment'],
               estimator_fit_params={'cat_features': categorical_features}
)

Learning rate set to 0.5
0:	learn: 0.5591869	total: 261ms	remaining: 4.96s
1:	learn: 0.5248715	total: 476ms	remaining: 4.29s
2:	learn: 0.5156534	total: 675ms	remaining: 3.83s
3:	learn: 0.5083190	total: 942ms	remaining: 3.77s
4:	learn: 0.5040869	total: 1.2s	remaining: 3.6s
5:	learn: 0.5005153	total: 1.67s	remaining: 3.9s
6:	learn: 0.4992875	total: 2.62s	remaining: 4.87s
7:	learn: 0.4977823	total: 3.65s	remaining: 5.47s
8:	learn: 0.4945390	total: 4.59s	remaining: 5.61s
9:	learn: 0.4932172	total: 5.03s	remaining: 5.03s
10:	learn: 0.4922512	total: 5.45s	remaining: 4.46s
11:	learn: 0.4913587	total: 5.89s	remaining: 3.93s
12:	learn: 0.4906069	total: 6.33s	remaining: 3.41s
13:	learn: 0.4898458	total: 6.59s	remaining: 2.83s
14:	learn: 0.4892095	total: 6.83s	remaining: 2.28s
15:	learn: 0.4874396	total: 7.07s	remaining: 1.77s
16:	learn: 0.4869179	total: 7.29s	remaining: 1.29s
17:	learn: 0.4862713	total: 7.55s	remaining: 839ms
18:	learn: 0.4857422	total: 7.77s	remaining: 409ms
19:	learn: 0.485026

SoloModel(estimator=<catboost.core.CatBoostClassifier object at 0x7fdc045cfd00>)

In [ ]:
uplift_best_cb = best_model.predict(X_test.drop(columns='treatment'))

In [ ]:
uplift_auc_score(y_true=y_test,
                             uplift=uplift_best_cb,
                             treatment=X_test['treatment'])

0.05129456670618262

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/Olymp_5_2/test.csv')

In [ ]:
df_test_new = df_test.copy()

In [ ]:
columns_to_fill_binary = ['feature_1','feature_2','cc_11','cc_12','feature_9','feature_13','bki_19','cc_15','feature_17','feature_21','feature_22','feature_24','feature_25','partner_24',
                          'feature_26']

for col in columns_to_fill_binary:
    df_test_new[col] = (df_test_new[col] >= 0.5).astype(int)
    # df_without_nan[col] = new_df[col].fillna(new_df['successful_utilization'])
    mode_value = df_test_new[col].mode()[0]
    df_test_new[col] = df_test_new[col].fillna(mode_value)


columns_to_fill_mode = ['mb_1','mb_2','application_2','mb_3','bki_6','application_5','partner_11','bki_12','feature_10','bki_14','partner_14','partner_17',
                        'feature_20','bki_27','application_7','bki_39','application_13','cc_22','application_16','application_15','cc_21','cc_1','feature_6','cc_2','cc_4','cc_6']

for col in columns_to_fill_mode:
    mode_value = df_test_new[col].mode()[0]  # Получаем моду (первое значение, если несколько)
    df_test_new[col] = df_test_new[col].fillna(mode_value)

columns_to_fill_median = ['application_1','cc_3','cc_5','cc_7','cc_8','application_2','partner_1','cc_8','mb_4','bki_1','graph_1',
                          'partner_2','bki_2','partner_3','application_3','application_4','feature_3','feature_4','bki_3','bki_4',
                          'bki_5','feature_5','mb_5','cc_9','partner_4','partner_5','cc_10','bki_7','partner_6','partner_7',
                          'bki_8','feature_7','partner_8','bki_9','bki_10','partner_9','partner_10','graph_2','feature_8',
                          'bki_11','mb_6','bki_13','feature_11','partner_12','partner_13']

excluded_columns = ['successful_utilization', 'treatment'] + columns_to_fill_binary + columns_to_fill_mode

for col in df_test_new.columns:
    if col not in excluded_columns and df_test_new[col].dtype in ['int64', 'float64']:
        median_value = df_test_new[col].median()
        df_test_new[col] = df_test_new[col].fillna(median_value)

In [ ]:
df_test_new['retro_date'] = pd.to_datetime(df_test_new['retro_date'])
df_test_new['year'] = df_test_new['retro_date'].dt.year
df_test_new['month'] = df_test_new['retro_date'].dt.month
df_test_new['day'] = df_test_new['retro_date'].dt.day

<ipython-input-48-6aec4f2989e1>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test_new['year'] = df_test_new['retro_date'].dt.year
<ipython-input-48-6aec4f2989e1>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test_new['month'] = df_test_new['retro_date'].dt.month
<ipython-input-48-6aec4f2989e1>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get

In [ ]:
df_test_new = df_test_new.drop('retro_date', axis=1)

In [ ]:
df_test_new_fi = df_test_new[columns_fi]

In [ ]:
uplift_best_cb = best_model.predict(df_test_new)

In [ ]:
uplift_best_cb.shape

(74313,)

In [ ]:
df_test_new['successful_utilization'] = uplift_best_cb

<ipython-input-51-997cc093cf0a>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test_new['successful_utilization'] = uplift_best_cb


In [ ]:
# Создаем DataFrame с индексами и предсказаниями
predictions_df = pd.DataFrame({
    'successful_utilization': uplift_best_cb      # Предсказанные значения uplift
})

# Сохраняем DataFrame в CSV файл
predictions_df.to_csv('cat_boost_8.csv', index=True)